#Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
!pip install faiss-cpu==1.8.0 -q
!pip install langchain==0.2.5 -q
!pip install langchain-community==0.2.5 -q
!pip install langchain-core===0.2.7 -q
!pip install langchain-openai==0.0.5 -q
!pip install langchain-text-splitters==0.2.1 -q
!pip install langsmith==0.1.81 -q
!pip install openai==1.10.0 -q
!pip install tiktoken==0.7.0 -q
!pip install pdfminer.six -q
!pip install langchain-google-genai -q

# Kütüphanelerin Eklenmesi


In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import time
import pickle
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader,PDFMinerLoader
from langchain_community.vectorstores import FAISS

from langchain.chains import  ConversationalRetrievalChain ,RetrievalQA
from langchain_community.llms import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate
from langchain.schema import HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from google.colab import userdata
from langchain.chains.question_answering import load_qa_chain

#import warnings
#warnings.filterwarnings("ignore")

# Rag uygulaması için gerekli PDF'lerin yüklenmesi ve Bölünmesi

In [ ]:
# Veri Setleri aşağıdadır.
# https://cbddo.gov.tr/arastirma-raporlari/
# https://cbddo.gov.tr/SharedFolderServer/Genel/File/TR-UlusalYZStratejisi2021-2025.pdf

In [ ]:
# Çalışma ortamında bir klasör oluştur !
kaynak = 'belgeler'
# Klasör içerideki dokumanları göster
dokuman_listesi = os.listdir(kaynak)
print(dokuman_listesi)


['TR-UlusalYZStratejisi2021-2025.pdf', '21.Chatbot-Uygulaması-ve-ChatGPT-Örneği-Değerlendirme-Raporu.pdf']


# Türkçe Metinlerde ,embeddings öncesi ön işleme aşaması

In [ ]:
bolunmus_dokuman = []

for i in dokuman_listesi:
    loader = PDFMinerLoader("belgeler/"+i)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200 , length_function = len)
    docs = text_splitter.split_documents(data)
    for k in range(len(docs)):
        bolunmus_dokuman.append(docs[k])
len(bolunmus_dokuman)


209

In [ ]:
bolunmus_dokuman[0:2]

[Document(page_content='a\n\n\x0cb\n\n\x0cc\n\n\x0cULUSAL YAPAY ZEKÂ STRATEJİSİ\n\n2021-2025\n\nhttps://www.cbddo.gov.tr/UYZS\n\nAğustos 2021\n\n\x0c\x0c“\n\nÜlkemiz adına \nyapay zekâ \nalanında \ninsan odaklı \nyeni bir atılım \nyapmanın \nzamanının \ngeldiğine \ninanıyoruz.\n\n“\n\nRecep Tayyip ERDOĞAN\nCumhurbaşkanı\n\nf\n\nU L U S A L   Y A P A Y   Z E K ÂS T R A T E J İ S İ\x0cRecep Tayyip ERDOĞAN\nCumhurbaşkanı\n\nÖn Söz\n\n21’inci yüzyılın ilk çeyreği; dijital veri ve algoritmalar ile dönüşen toplumsal, \nekonomik  ve  siyasi  yapıların  analog  döneme  ait  kavramlar  ile  olan \nuyumsuzluklarının gün yüzüne çıktığı bir dönem olmaktadır. Hızla gelişen \nyapay zekâ destekli sistemlerin üretim süreçleri, meslekler, gündelik yaşam \nve kurumsal yapılar üzerindeki köklü dönüştürücü etkisi insanoğlunu yeni \nbir  çağın  eşiğine  getirmiştir.  Uygulama  alanı  giderek  artan  yapay  zekâ \nteknolojilerinin,  küresel  ekonomik  yapı  üzerinde  İnternet  devriminden \ndaha büyük bir e

In [ ]:
bolunmus_dokuman = []

def metin_önişleme(documents):
    for i in range(len(documents)):
        documents[i].page_content = re.sub(r'\.{2,}', '.', documents[i].page_content)
        documents[i].page_content = documents[i].page_content.replace("\n"," ")
        documents[i].page_content = re.sub(r'\s{3,}', " ",documents[i].page_content)
    return documents

for i in dokuman_listesi:
    loader = PDFMinerLoader("belgeler/"+i)
    data = loader.load()
    data = metin_önişleme(data)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200 , length_function = len)
    docs = text_splitter.split_documents(data)
    for k in range(len(docs)):
        bolunmus_dokuman.append(docs[k])
len(bolunmus_dokuman)


189

In [ ]:
(209-189) /209*100

9.569377990430622

In [ ]:
bolunmus_dokuman[0:2]

[Document(page_content='a b c ULUSAL YAPAY ZEKÂ STRATEJİSİ  2021-2025  https://www.cbddo.gov.tr/UYZS  Ağustos 2021 “  Ülkemiz adına  yapay zekâ  alanında  insan odaklı  yeni bir atılım  yapmanın  zamanının  geldiğine  inanıyoruz.  “  Recep Tayyip ERDOĞAN Cumhurbaşkanı  f  U L U S A L Y A P A Y Z E K ÂS T R A T E J İ S İ\x0cRecep Tayyip ERDOĞAN Cumhurbaşkanı  Ön Söz  21’inci yüzyılın ilk çeyreği; dijital veri ve algoritmalar ile dönüşen toplumsal,  ekonomik  ve  siyasi  yapıların  analog  döneme  ait  kavramlar  ile  olan  uyumsuzluklarının gün yüzüne çıktığı bir dönem olmaktadır. Hızla gelişen  yapay zekâ destekli sistemlerin üretim süreçleri, meslekler, gündelik yaşam  ve kurumsal yapılar üzerindeki köklü dönüştürücü etkisi insanoğlunu yeni  bir  çağın  eşiğine  getirmiştir.  Uygulama  alanı  giderek  artan  yapay  zekâ  teknolojilerinin,  küresel  ekonomik  yapı  üzerinde  İnternet  devriminden  daha büyük bir etki yaratması beklenmektedir. Sibernetik  alanının  kurucusu  kabul  edil

#Bolunmus Dokumanların Kaydedilmesi

In [ ]:
# Bölünmüş dokumanların kaydedilmesi
with open("documents_chunk_1000_overlap_100", "wb") as fp:
    pickle.dump(bolunmus_dokuman, fp)
# Bölünmüş dokumanların tekrar çağırılması
with open("documents_chunk_1000_overlap_100", "rb") as fp: #Unpickling
    kaydedilmis_belgeler = pickle.load(fp)

In [ ]:
#kaydedilmis_belgeler

# Embeddings

## Gemini

In [ ]:
# API key için https://aistudio.google.com/app/apikey
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Türkçe desteği var ! Daha fazla bilgi için aşağıdaki siteyi ziyaret edebilirsiniz.
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api
embeddings =  GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
text = "Bu bir test cümlesidir."
query_result = embeddings.embed_query(text)
print(len(query_result))
query_result[:10]

768


[-0.003097370732575655,
 0.05171157419681549,
 -0.028281938284635544,
 -0.0732334777712822,
 0.022028723731637,
 0.03542127460241318,
 0.016467075794935226,
 -0.004859305918216705,
 0.004303519614040852,
 -0.02327588014304638]

In [ ]:
text = "."
query_result = embeddings.embed_query(text)
print(len(query_result))
query_result[:10]

768


[0.03799588605761528,
 0.034429799765348434,
 -0.02474670298397541,
 -0.004422950092703104,
 -0.00876795407384634,
 0.05222013220191002,
 0.019114097580313683,
 0.008022915571928024,
 0.057971104979515076,
 0.024201415479183197]

# Vector Veritabanının Oluşturulması

In [ ]:
vector_veritabanı = FAISS.from_documents(bolunmus_dokuman, embedding=embeddings)
vector_veritabanı.save_local("gemini_embedding_vectordb")

#Langchain ile Rag Uygulaması - Gemini

In [ ]:
chain_type="stuff" #  map_reduce, refine , map-rerank # Detaylar için https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a
search_type = "similarity" #'similarity', 'similarity_score_threshold', 'mmr' detaylar için #https://medium.com/tech-that-works/maximal-marginal-relevance-to-rerank-results-in-unsupervised-keyphrase-extraction-22d95015c7c5
prompt_template = """
    1- Aşağıdaki bağlamları kullanarak soruyu yanıtlayın.
    2- Sağlanan bağlamdan en ayrıntılı şekilde soruyu yanıtlayın.
    3- En fazla 5 cümle kullanın.
    4- Cevap sağlanan bağlamda yoksa, "Üzgünüm cevabı bulamadım..." demelisiniz.
    5- Yanıt Türkçe olmalıdır.
    Context:\n {context}\n
    Question: \n{question}\n
  """

vb = FAISS.load_local("/content/gemini_embedding_vectordb", embeddings,allow_dangerous_deserialization=True)

retriever = vb.as_retriever(search_type=search_type, search_kwargs={"k": 5})
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro",  temperature=0)
prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
retrievalQA = RetrievalQA.from_llm(llm=llm, retriever=retriever,prompt=prompt)


#Gemini-Pro-1.0 ile yapılan RAG uygulamasının Test Edilmesi

In [ ]:
# Sorunun cevabı Yapay zeka stratejisi dökumanında sayfa 7 'de detaylı verilmiştir. Teyit için kontrol edilebilir.
retrievalQA.run("Türkiye'nin yapay zeka stratejisini özetle")

"Türkiye'nin Ulusal Yapay Zekâ Stratejisi, 2021-2025 yılları arasında ülkenin yapay zeka alanındaki çalışmalarını ortak bir zemine oturtmayı amaçlamaktadır. Strateji, yapay zeka uzmanlarını yetiştirmeyi, araştırma ve yenilikçiliği desteklemeyi, kaliteli veriye erişimi genişletmeyi, sosyoekonomik uyumu hızlandıracak düzenlemeler yapmayı, uluslararası iş birliklerini güçlendirmeyi ve yapısal ve iş gücü dönüşümünü hızlandırmayı hedeflemektedir."

In [ ]:
# Sorunun cevabı sayfa 9 'da detaylı verilmiştir. Teyit için kontrol edilebilir.
retrievalQA.run("UYZS’nin uygulama döneminin sonu olan 2025 yılında ulaşılması öngörülen üst seviye hedefleri nelerdir?")

"* YZ alanının GSYH'ye katkısı %5'e yükseltilecektir.\n* Merkezî ve yerel yönetim kamu kurum ve kuruluşlarında YZ alanında istihdam 1.000 kişiye çıkarılacaktır.\n* YZ alanında istihdam 50.000 kişiye çıkarılacaktır.\n* YZ alanında lisansüstü düzeyde mezun sayısı 10.000 kişiye çıkarılacaktır.\n* Uluslararası YZ endekslerindeki sıralamalarda Türkiye'nin ilk 20 ülke arasında yer alması sağlanacaktır."

In [ ]:
# Sorunun cevabı Chatbot uygulaması doumanında sayfa 11 'de detaylı verilmiştir. Teyit için kontrol edilebilir.
retrievalQA.run("Kural Tabanlı Chatbot nedir ?")

"Kural tabanlı chatbot'lar, kullanıcıların sorgularını kalıp eşleşmeleri arayarak yanıtlayan ve bilinen herhangi bir kalıbı içermeyen cümlelerle karşılaştıklarında yanlış yanıtlar veren chatbot türüdür."

In [ ]:
# Sorunun cevabı Chatbot uygulaması doumanında sayfa 22 'de detaylı verilmiştir. Teyit için kontrol edilebilir.
retrievalQA.run("Chatbot’ların İşletmelere Sağladığı Faydaları madde madde verir misin?")

'* Maliyet tasarrufu\n* Web sitesi ziyaretçilerine içerik tabanlı yapay zeka tabanlı destek sunma\n* Müşteri memnuniyetini artırma\n* Müşteri hizmetleri departmanının yükünü azaltma\n* Müşterilerle 7/24 iletişim kurma'

In [ ]:
# Sorunun cevabı Chatbot uygulaması doumanında sayfa 47 'de detaylı verilmiştir. Teyit için kontrol edilebilir.
retrievalQA.run("ChatGPT'yi yasaklayan okulların adı nedir?")

'New York Devlet Okulları, Seattle Devlet Okulları, Longview Okul Bölgesi, Kelso Okul Bölgesi, Baltimore Devlet Okulları, Los Angeles Okul Bölgesi, Fairfax Devlet Okulları, Montgomery Alabama Devlet Okulları'

# Langchain ile Rag Uygulaması OpenAI